In [1]:
spark

In [2]:
dir0 = '/home/cloudera/2.kkbox_churn/data01/from_raw_transactions-v1/feature_creation/'

In [16]:
# subdir = '32.features_categorical/'
subdir = '33.features_continuous/'

divisions = ['msno_a-n', 'msno_A-N',  'msno_non-letter', 'msno_o-z', 'msno_O-Z']
division = divisions[4]

# items = ['actual_paid', 'auto_renew', 'discount', 'list_price', 'pay_method', 'plan_days']
items = ['actual_paid', 'discount', 'list_price']

dfs = []
for it in items:
    infile = dir0+subdir+division+'/'+it
    df = spark.read.format('csv').option('header', 'true').load(infile)
    dfs.append(df)
    
df_join = dfs[0]
for df in dfs[1:]:
    df1 = df.withColumnRenamed('msno', 'msno_')
    df_join = df_join.join(df1, df_join['msno']==df1['msno_'], 'inner').drop('msno_')
    
outfile = dir0+subdir+division+'/'+'all-features'
df_join.write.format('csv').option('header','true').save(outfile)

In [24]:
infile = dir0+'30.features_1'
df30 = spark.read.format('csv').option('header','true').load(infile) \
       .drop('gaps')

In [30]:
subdirs = ['31.features_latest/', '32.features_categorical/', '33.features_continuous/']

divisions = ['msno_a-n', 'msno_A-N',  'msno_non-letter', 'msno_o-z', 'msno_O-Z']
division = divisions[4]

###
df_join = df30
for subdir in subdirs:
    infile = dir0+subdir+division
    if not subdir.startswith('31.'):
        infile += '/'+'all-features'
    df = spark.read.format('csv').option('header','true').load(infile) \
         .withColumnRenamed('msno', 'msno_')
    df_join = df_join.join(df, df_join['msno']==df['msno_'], 'inner').drop('msno_')
    
outfile = dir0 + '34.all_trans_features/' + division
df_join.write.format('csv').option('header','true').save(outfile)

In [8]:
subdir = '34.all_trans_features/'
divisions = ['msno_a-n', 'msno_A-N',  'msno_non-letter', 'msno_o-z', 'msno_O-Z']

dfs = []
for division in divisions:
    infile = dir0 + subdir + division
    df = spark.read.format('csv').option('header','true').load(infile)
    dfs.append(df)
    
df_union = dfs[0]
for df in dfs[1:]:
    df_union = df_union.union(df)
    
outfile = dir0 + subdir + 'all-members'
df_union.coalesce(8).write.format('csv').option('header','true').save(outfile)